In [25]:
import pandas as pd
import re
import spacy
from tqdm import tqdm

tqdm.pandas()

In [26]:
en_train_path = "../data/processed/processed_CT24_checkworthy_english/sentence_split_processed_train.tsv"  # noqa: E501
du_train_path = "../data/processed/processed_CT24_checkworthy_dutch/sentence_split_processed_dutch_train.tsv"  # noqa: E501
es_train_path = "../data/processed/processed_CT24_checkworthy_spanish/sentence_split_processed_spanish_train.tsv"  # noqa: E501
ar_train_path = "../data/processed/processed_CT24_checkworthy_arabic/sentence_split_processed_arabic_train.tsv"  # noqa: E501

en_train = pd.read_csv(en_train_path, sep="\t")
du_train = pd.read_csv(du_train_path, sep="\t")
es_train = pd.read_csv(es_train_path, sep="\t")
ar_train = pd.read_csv(ar_train_path, sep="\t")

In [27]:
import re


def correct_tags(text):
    pattern = r"(<s>)+|(<\/s>)+"

    def replace_func(match):
        tag = match.group(0)
        if "<s>" in tag:
            return "<s>"
        else:
            return "</s>"

    corrected_text = re.sub(pattern, replace_func, text)
    return corrected_text

In [28]:
# https://spacy.io/models
# python -m spacy download en_core_web_trf
# python -m spacy download nl_core_news_lg
# python -m spacy download es_dep_news_trf
# python -m spacy download xx_sent_ud_sm

spacy_models = {
    "en": "en_core_web_sm",
    "nl": "nl_core_news_lg",
    "es": "es_core_news_sm",
    "ar": "xx_sent_ud_sm",
}

nlp_models = {lang: spacy.load(model) for lang, model in spacy_models.items()}

for dataset, dataset_path, lang in [
    (en_train, en_train_path, "en"),
    (du_train, du_train_path, "nl"),
    (es_train, es_train_path, "es"),
    (ar_train, ar_train_path, "ar"),
]:

    def add_stop_token(text: str) -> str:
        # if lang not in nlp_models:
        #     raise ValueError("Ugyldig språk")

        # nlp = nlp_models[lang]
        # text = text.replace("#", " ")
        # doc = nlp(text)
        # sentences = []

        # for sent in doc.sents:
        #     if re.search(r"\b(?:[A-Z]\.)+\b", sent.text):
        #         next_sent = doc[sent.end].text if sent.end < len(doc) else ""
        #         combined_sent = sent.text + next_sent
        #         sentences.append(f"<s> {combined_sent.strip()} </s>")
        #     else:
        #         sentences.append(f"<s> {sent.text.strip()} </s>")

        # sentences = " ".join(sentences)
        # sentences = correct_tags(sentences)

        # sentences = re.sub(r"<s>(.*?<s>.*?</s>.*?)</s>", r"\1", sentences)
        # sentences = re.sub(r"<s>(.*?</s>.*?)<s>", r"\1", sentences)
        text = text.replace("\n", " ")
        text = text.replace("\t", " ")
        text = text.replace("<s>", "")
        text = text.replace("</s>", "")
        text = re.sub(r" +", " ", text)
        return text

    def correct_tags(text: str) -> str:
        pattern = r"(<s>)+|(<\/s>)+"

        def replace_func(match):
            return "<s>" if "<s>" in match.group(0) else "</s>"

        corrected_text = re.sub(pattern, replace_func, text)
        return corrected_text

    dataset["tweet_text"] = dataset["tweet_text"].progress_apply(
        lambda x: x.replace("<s>", "").replace("</s>", "")
    )
    dataset["tweet_text"] = dataset["tweet_text"].progress_apply(add_stop_token)

    display(dataset)

    dataset.to_csv(dataset_path, sep="\t", index=False)

100%|██████████| 22494/22494 [00:00<00:00, 283057.29it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,class_label_encoded,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,30313,And so I know that this campaign has caused so...,No,118,NaN,NaN,100+,0,0,0.0,0
1,19099,"Now, let's balance the budget and protect Medi...",No,94,NaN,NaN,91-100,0,0,0.0,0
2,33964,I'd like to mention one thing.,No,30,NaN,NaN,21-30,0,0,0.0,0
3,16871,I must remind him the Democrats have controlle...,Yes,124,NaN,NaN,100+,1,0,0.0,0
4,13150,And to take a chance - now be - and not make e...,No,160,NaN,NaN,100+,0,0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
22489,29631,"It would be squandered, too, believe me.",No,42,NaN,NaN,41-50,0,0,0.0,0
22490,7136,We're not allowed to vote on it.,Yes,32,NaN,NaN,31-40,1,0,0.0,0
22491,181,More Americans at work today than any time in ...,Yes,96,NaN,NaN,91-100,1,0,0.0,0
22492,12863,We indicated at that time that we were not goi...,Yes,73,NaN,NaN,71-80,1,0,0.0,0


100%|██████████| 994/994 [00:00<00:00, 230772.62it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,class_label_encoded,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,1240603314980392966,"#SamenTegenCorona:applaus voor zorghelden, hui...",No,98,SamenTegenCorona,NaN,91-100,0,1,0.0,0
1,1238094774453833730,Kabinet ondersteunt ondernemer in Corona-crisi...,No,69,coronavirusNederland,NaN,61-70,0,1,0.0,1
2,1238154603583156225,Heropening van het @<USER> in #Oosterbeek uitg...,No,84,Oosterbeek Airborne Covid19 coronavirus,airbornemuseum,81-90,0,4,0.0,3
3,1239152526026518534,Aantal restaurants in#groningen nu op slot.#bl...,Yes,56,groningen blijfthuis,NaN,51-60,1,2,0.0,2
4,1244341963480076290,Nederland heeft het niet in de hand.Onbetrouwb...,Yes,220,RIVM COVID2019NL Coronavirusnl RIVM coronapati...,NaN,100+,1,5,0.0,4
...,...,...,...,...,...,...,...,...,...,...,...
989,1253322004544892937,Door Covid-19 zetten organisaties massaal in o...,No,249,bedrijfscontinuïteit IT wayofworking,NaN,100+,0,3,0.0,0
990,1240614429127041024,Ons crisisteam #Corona gaat locaties af om onz...,No,257,Corona vitaleberoepen Waardering noodopvang ki...,NaN,100+,0,7,0.0,3
991,1305431406647087106,@<USER>En die Covid-19 piek bleef nog laag doo...,Yes,214,NaN,demandieallesk2 aajbrouwer volkskrant,100+,1,0,0.0,0
992,1268546629935804418,Vandaag op onze website: nieuwe versies van co...,No,234,NaN,gkv ngk_nl,100+,0,0,0.0,0


100%|██████████| 16313/16313 [00:00<00:00, 164934.71it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,class_label_encoded,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,1219231756064296962,¿Tienen derecho los padres a decidir muchas cu...,</s><s>No,42,[],[],41-50,0,0,0.0,1
1,1190251970101166080,El tipo efectivo del impuesto de sociedades (I...,</s><s>Yes,40,[],[],31-40,1,0,0.0,0
2,1192022562240380929,Escuelas infantiles públicas y gratuitas de 0 ...,No,49,['#aversivoyaserdeUP'],[],41-50,0,1,0.0,0
3,1271859669406035969,Derrumbe de la estrategia judicial creada cont...,</s><s>Yes,40,[],['@pablocasado_'],31-40,1,0,0.0,0
4,1272885337472339969,El Gobierno aprueba el Fondo COVID-19 de 16.00...,Yes,42,"['#NadieSeQuedaAtrás', '#CMin']",['@mjmonteroc:'],41-50,1,2,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...
16308,189627,Cuando Inés Arrimadas le contó dijo a Pablo Ca...,Yes,72,[],[],71-80,1,0,0.0,4
16309,222938,Al menos yo ser presidente no tocar ese asunto...,No,17,[],[],11-20,0,0,0.0,4
16310,207681,En memoria de quienes durante años nos han tra...,No,10,[],[],0-10,0,0,0.0,0
16311,330393,Enrique López Muchas gracias por estar con nos...,No,16,[],[],11-20,0,0,0.0,3


100%|██████████| 7333/7333 [00:00<00:00, 159921.96it/s]


,tweet_id,tweet_text,class_label,text_length,hashtags,mentions,text_length_category,class_label_encoded,hashtags_frequency,hashtags_sentiment,hashtags_topics
0,1221949644554587904,"وأي خيانة أكبر من خيانة الدين , الوطن , الهوية...",No,189,صفقة_القرن القدس_عاصمة_فلسطين_الأبدية,NaN,100+,0,2,0.0,4
1,1222030473385345024,صباح الخير على فلسطين وقدسها وشعبها ومهجرينها ...,No,169,القدس_عاصمة_فلسطين_الأبديه تسقط_صفقة_الوهم,NaN,100+,0,2,0.0,1
2,1222035929105338368,ذنبگ أنگ جميله گ يوسف وخانگ العالم گ أاخواته ص...,No,98,صفقة_القرن_لن_تمر,NaN,91-100,0,1,0.0,1
3,1222048121145962496,لا يلزمك ان تكون فلسطينياً لتحب فلسطين، حُبها ...,No,168,صباح_الخير القدس_عاصمة_فلسطين_الأبدية,NaN,100+,0,2,0.0,4
4,1222053294266372096,#مناصرون رحل البروفسور زيك فريد فوغل .رئيس قسم...,No,270,مناصرون صفقة_القرن_لن_تمر,NaN,100+,0,2,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...
7328,1366469668089827328,وزير الصحة: ربط قبول المشاركين في موسم #الحج ب...,No,106,الحج كورونا,NaN,100+,0,2,0.0,2
7329,1296531196462202880,#روسيا تخطط لتصدير اللقاح ضد #كورونا في نهاية ...,Yes,58,روسيا كورونا,NaN,51-60,1,2,0.0,0
7330,1293180035663241216,"#روسيا تطلق اسم ""سبوتنيك 5"" على أول لقاح فى ال...",No,75,روسيا فيروس_كورونا كورونا,NaN,71-80,0,3,0.0,0
7331,1242563204661809152,#عاجل برئاسة #خادم_الحرمين_الشريفين ..عقد القم...,No,147,عاجل خادم_الحرمين_الشريفين مجموعة_العشرين فيرو...,NaN,100+,0,5,0.0,1
